## **Análise filogenética**

O código  realiza um alinhamento múltiplo de sete sequências, selecionando os melhores alinhamentos de sequências homólogas. Em seguida, processa o cálculo da matriz, utilizando como base o alinhamento obtido com a matriz modelo 'BLOSUM62'. Por fim, concontroi árvores filogenéticas pelos métodos UPGMA e NJ para cada gene em análise.


In [1]:
from Bio import SeqIO, AlignIO, Phylo
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from collections import Counter
import os
import matplotlib.pyplot as plt

def process_gene(gene_name, input_fasta):
    # Criar diretório de saída
    output_dir = "Resultados da Análise Filogenética"
    os.makedirs(output_dir, exist_ok=True)

    # Leitura do arquivo fasta
    with open(input_fasta) as handle:
        records = list(SeqIO.parse(handle, format='fasta'))

    # Garantir que temos exatamente 7 sequências
    if len(records) < 7:
        raise ValueError(f"O arquivo FASTA deve conter pelo menos 7 sequências. Foram encontradas: {len(records)}")

    # Extrair as primeiras 7 sequências e usa o nome da espécie como ID
    seq_records = []
    for record in records[:7]:
        # Extrair o nome da espécie da descrição
        species = record.description.split('Species: ')[-1].strip()
        # Criar um novo SeqRecord com o nome da espécie como ID
        new_record = SeqRecord(record.seq, id=species, description='')
        seq_records.append(new_record)

    # Criar o alinhamento
    alignment = MultipleSeqAlignment(seq_records)

    # Verificar e renomear IDs duplicados no MSA
    unique_ids = set()
    for i, record in enumerate(alignment):
        original_id = record.id
        while record.id in unique_ids:
            print(f"ID duplicado encontrado: {record.id}. Renomeando...")
            record.id = f"{original_id}_{i}"
        unique_ids.add(record.id)

    # Salvar o alinhamento
    alignment_file = os.path.join(output_dir, f'{gene_name}_alignment.fasta')
    AlignIO.write(alignment, alignment_file, 'fasta')

    # Calcular matriz de distância
    calculator = DistanceCalculator('blosum62')
    dm = calculator.get_distance(alignment)
    
    # Imprimir a matriz de distância
    print(f"Matriz de Distância:\n{dm}")

    # Construir árvores
    constructor = DistanceTreeConstructor()
    upgma_tree = constructor.upgma(dm)
    nj_tree = constructor.nj(dm)

    # Remover rótulos 'Inner' dos clades e destacar-los com o género "Homo"
    for tree in [upgma_tree, nj_tree]:
        for clade in tree.find_clades():
            if 'Inner' in (clade.name or ''):
                clade.name = ""
            if 'Homo' in (clade.name or ''):
                clade.color = 'orange'
                clade.name = '*** ' + clade.name + ' ***'  # Marca o clade

    # Guardar árvores
    Phylo.write(upgma_tree, os.path.join(output_dir, f'{gene_name}_upgma_tree.nwk'), 'newick')
    Phylo.write(nj_tree, os.path.join(output_dir, f'{gene_name}_nj_tree.nwk'), 'newick')

    # Processar e desenhar árvores
    for tree, tree_type in [(upgma_tree, 'UPGMA'), (nj_tree, 'NJ')]:
        plt.figure(figsize=(15, 10))
        Phylo.draw(tree, do_show=False)
        plt.title(f'{gene_name} - {tree_type} Tree')
        tree_img = os.path.join(output_dir, f'{gene_name}_{tree_type}_tree.png')
        plt.savefig(tree_img)
        plt.close()

    print(f"Processamento concluído para o gene {gene_name}. Resultados guardado em '{output_dir}'.")


#### **1: Gene ptsP**

In [2]:
gene_name = "ptsp"
input_fasta = "genes/ptsP_blast.fasta"

process_gene(gene_name, input_fasta)

ID duplicado encontrado: Faecalibacterium prausnitzii. Renomeando...
ID duplicado encontrado: Faecalibacterium prausnitzii. Renomeando...
ID duplicado encontrado: Faecalibacterium prausnitzii. Renomeando...
ID duplicado encontrado: Faecalibacterium prausnitzii. Renomeando...
ID duplicado encontrado: Faecalibacterium prausnitzii. Renomeando...
Matriz de Distância:
Faecalibacterium prausnitzii    0.000000
Faecalibacterium prausnitzii_1  0.001080    0.000000
Faecalibacterium prausnitzii_2  0.001080    0.002159    0.000000
Faecalibacterium prausnitzii_3  0.001439    0.002519    0.002519    0.000000
Faecalibacterium prausnitzii_4  0.002157    0.003236    0.003236    0.003596    0.000000
Faecalibacterium sp.    0.001439    0.000360    0.002519    0.002879    0.003596    0.000000
Faecalibacterium prausnitzii_6  0.001799    0.002879    0.002879    0.003239    0.003955    0.003239    0.000000
    Faecalibacterium prausnitzii    Faecalibacterium prausnitzii_1  Faecalibacterium prausnitzii_2  Fae

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

#### **2: Gene ButyrylCoA**

In [3]:
gene_name = "butyrylCoA"
input_fasta = "genes/butyrylCoA_blast.fasta"

process_gene(gene_name, input_fasta)

ID duplicado encontrado: Faecalibacterium. Renomeando...
ID duplicado encontrado: Faecalibacterium prausnitzii. Renomeando...
ID duplicado encontrado: Faecalibacterium prausnitzii. Renomeando...
Matriz de Distância:
Faecalibacterium    0.000000
Faecalibacterium_1  0.002124    0.000000
Faecalibacterium prausnitzii    0.002123    0.004246    0.000000
Faecalibacterium prausnitzii M21/2  0.000000    0.002124    0.002123    0.000000
Faecalibacterium sp.    0.003398    0.001700    0.005520    0.003398    0.000000
Faecalibacterium prausnitzii_5  0.003398    0.001276    0.005520    0.003398    0.002975    0.000000
Faecalibacterium prausnitzii_6  0.003398    0.001276    0.005520    0.003398    0.002975    0.002551    0.000000
    Faecalibacterium    Faecalibacterium_1  Faecalibacterium prausnitzii    Faecalibacterium prausnitzii M21/2  Faecalibacterium sp.    Faecalibacterium prausnitzii_5  Faecalibacterium prausnitzii_6
Processamento concluído para o gene butyrylCoA. Resultados guardado em 'Re

<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>